# Scripts to download data for AE KB

In [12]:
import requests
import json
from pathlib import Path


PATH_DATA = Path('./data/askextension_kb')

In [14]:
START_DATE = '2012-01-01'
END_DATE = '2013-12-31'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-{END_DATE[:4]}.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2012-01-01 - 2013-12-31
Number of tickets - 11900
First and last 5 items:
(['2012-12-03 15:53:47', '2012-12-03 16:05:27', '2012-12-03 16:42:05', '2012-12-03 16:59:02', '2012-12-03 18:39:40'], ['2013-12-30 23:59:38', '2013-12-31 00:27:34', '2013-12-31 02:12:31', '2013-12-31 03:01:19', '2013-12-31 04:48:08'])


In [15]:
START_DATE = '2014-01-01'
END_DATE = '2015-12-31'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-{END_DATE[:4]}.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2014-01-01 - 2015-12-31
Number of tickets - 37692
First and last 5 items:
(['2014-01-01 15:37:32', '2014-01-01 18:46:03', '2014-01-01 20:37:50', '2014-01-01 23:04:06', '2014-01-02 00:03:35'], ['2015-12-31 02:08:57', '2015-12-31 02:19:22', '2015-12-31 03:12:52', '2015-12-31 03:19:36', '2015-12-31 06:36:42'])


In [16]:
START_DATE = '2016-01-01'
END_DATE = '2017-12-31'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-{END_DATE[:4]}.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2016-01-01 - 2017-12-31
Number of tickets - 45176
First and last 5 items:
(['2016-01-01 13:17:43', '2016-01-01 13:43:12', '2016-01-01 14:43:17', '2016-01-01 15:53:35', '2016-01-01 16:25:43'], ['2017-12-30 20:52:54', '2017-12-30 23:41:33', '2017-12-31 01:19:26', '2017-12-31 05:46:57', '2017-12-31 05:53:04'])


In [17]:
START_DATE = '2018-01-01'
END_DATE = '2019-12-31'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-{END_DATE[:4]}.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2018-01-01 - 2019-12-31
Number of tickets - 37640
First and last 5 items:
(['2018-01-01 17:14:58', '2018-01-01 18:06:03', '2018-01-01 18:27:29', '2018-01-01 18:35:18', '2018-01-01 19:03:58'], ['2019-12-31 00:13:25', '2019-12-31 01:19:29', '2019-12-31 01:21:52', '2019-12-31 02:08:32', '2019-12-31 02:34:50'])


In [23]:
START_DATE = '2020-01-01'
END_DATE = '2020-06-30'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-1.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2020-01-01 - 2020-06-30
Number of tickets - 14765
First and last 5 items:
(['2020-01-01 15:37:17', '2020-01-01 16:58:47', '2020-01-01 17:49:07', '2020-01-01 18:02:55', '2020-01-01 18:29:48'], ['2020-06-30 03:19:53', '2020-06-30 04:08:38', '2020-06-30 04:12:57', '2020-06-30 05:23:17', '2020-06-30 08:42:31'])


In [24]:
START_DATE = '2020-07-01'
END_DATE = '2020-12-31'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-2.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2020-07-01 - 2020-12-31
Number of tickets - 18930
First and last 5 items:
(['2020-07-01 10:40:43', '2020-07-01 11:26:27', '2020-07-01 11:32:42', '2020-07-01 11:38:47', '2020-07-01 11:47:15'], ['2020-12-31 03:11:35', '2020-12-31 04:06:36', '2020-12-31 04:26:52', '2020-12-31 04:28:03', '2020-12-31 04:35:12'])


In [26]:
START_DATE = '2021-01-01'
END_DATE = '2021-06-30'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-1.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2021-01-01 - 2021-06-30
Number of tickets - 20824
First and last 5 items:
(['2021-01-01 14:23:58', '2021-01-01 15:15:28', '2021-01-01 16:08:26', '2021-01-01 16:19:09', '2021-01-01 17:48:56'], ['2021-06-30 05:14:41', '2021-06-30 05:18:15', '2021-06-30 08:51:38', '2021-06-30 09:49:53', '2021-06-30 10:18:24'])


In [27]:
START_DATE = '2021-07-01'
END_DATE = '2021-12-31'
print(f'Dates: {START_DATE} - {END_DATE}')
URL = f'https://qa.osticket.eduworks.com/api/knowledge/{START_DATE}/{END_DATE}'
PATH_SAVE = Path.joinpath(PATH_DATA, f'{START_DATE[:4]}-2.json')

response = requests.get(URL)

res = response.json()
print(f'Number of tickets - {len(res)}')
dates = [r['created'] for r in res]
print(f'First and last 5 items:\n{dates[:5], dates[-5:]}')

with open(PATH_SAVE, 'w') as f:
    json.dump(res, f)

Dates: 2021-07-01 - 2021-12-31
Number of tickets - 13821
First and last 5 items:
(['2021-07-01 11:18:01', '2021-07-01 11:23:58', '2021-07-01 12:08:50', '2021-07-01 12:17:32', '2021-07-01 12:32:56'], ['2021-11-05 17:59:08', '2021-11-05 18:23:53', '2021-11-11 21:12:34', '2021-11-11 23:20:29', '2021-11-19 23:30:15'])
